In [ ]:
import azureml.core
import azureml.dataprep
from azureml.automl.core.constants import FeaturizationConfigMode
print(azureml.dataprep.__version__)
import azureml.core
print(azureml.core.VERSION)


In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core import Datastore

sql_datastore = Datastore.get(workspace=ws, datastore_name="ado_sql_datastore")


In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

label ="MitigationScore"
query_string = 'SELECT MitigationDescription, MitigationScore FROM FeedbackItems'

query = DataPath(sql_datastore, query_string)
feedback_sql_ds = Dataset.Tabular.from_sql_query(query)

feedback_sql_ds.register(workspace=ws,
                         name="ai_ag_ado_feedack_mitigation_score",
                         description = "Feedback from Azure DevOps",
                         create_new_version=True)

In [ ]:
def split_dataset(dataset):
    # Split the dataset into train and test datasets
    train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

    # Register the train dataset with your workspace
    train_data.register(workspace = ws, 
                        name = 'ai_ag_ado_feedack_mitigation_score_train_dataset',
                        description = 'Feedback from Azure DevOps training data',
                        create_new_version=True)

    # Register the test dataset with your workspace
    test_data.register(workspace = ws, 
                       name = 'ai_ag_ado_feedack_mitigation_score_test_dataset', 
                       description = 'Feedback from Azure DevOps test data',
                       create_new_version=True)
    return train_data, test_data
    
train_data, test_data = split_dataset(feedback_sql_ds)    

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             training_data = train_data,
                             label_column_name = label,
                             verbosity = logging.INFO,
                             enable_early_stopping = True,
                             max_concurrent_iterations = 4,
                             max_cores_per_iteration = -1,
                             n_cross_validations = 5,
                             primary_metric ='r2_score',
                             preprocess=True,
                             featurization = FeaturizationConfigMode.Auto,
                             enable_tf = True
                            )

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "mitigation-score-dc-sql")

local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, fitted_model = local_run.get_output()

model = best_run.register_model(model_name='best_sql_dc_mitigation_score', model_path='./outputs/model.pkl')

In [ ]:
print("Registered model:\n --> Name: {}\n --> Version: {}\n --> URL: {}".format(model.name, model.version, model.url))